In [56]:
import torch
import torch.nn as nn
import torchvision.models as models
import pandas as pd
import os
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

class MultiModalModel(nn.Module):
    def __init__(self, tabular_input_dim):
        super(MultiModalModel, self).__init__()
        self.cnn = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
        cnn_out_features = self.cnn.fc.in_features
        self.cnn.fc = nn.Identity()
        
        self.mlp = nn.Sequential(
            nn.Linear(tabular_input_dim, 64),
            nn.ReLU(),
            nn.Dropout(0.2)
        )
        self.fusion = nn.Sequential(
            nn.Linear(512 + 64, 128),
            nn.ReLU(),
            nn.Linear(128, 1)
        )

    def forward(self, image, tabular):
        img_feats = self.cnn(image)
        tab_feats = self.mlp(tabular)
        combined = torch.cat((img_feats, tab_feats), dim=1)
        return self.fusion(combined)

In [57]:
from torch.utils.data import DataLoader
train_ds = MultimodalDataset("data/processed_train.csv", "house_images", transform=transform)
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)

In [ ]:
class MultimodalDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        self.data = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        img_path = os.path.join(self.img_dir, f"{int(row['id'])}.jpg")
        image = Image.open(img_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        tabular = torch.tensor(row.drop(['id', 'price_log']).values, dtype=torch.float)
        target = torch.tensor(row['price_log'], dtype=torch.float).unsqueeze(0)
        
        return image, tabular, target

In [ ]:
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MultiModalModel(tabular_input_dim=17).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)


epochs = 20
for epoch in range(epochs):
    model.train()
    total_loss = 0
    loop = tqdm(train_loader, total=len(train_loader), leave=True)
    loop.set_description(f"Epoch [{epoch+1}/{epochs}]")

    for images, tabular, targets in loop:
        images, tabular, targets = images.to(device), tabular.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(images, tabular)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        loop.set_postfix(loss=loss.item())

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1} complete. Average Loss: {avg_loss:.4f}")
torch.save(model.state_dict(), "data/multimodal_model.pth")
print("Model saved successfully!")

Epoch [1/20]: 100%|██████████| 402/402 [02:45<00:00,  2.43it/s, loss=0.18]  


Epoch 1 complete. Average Loss: 12.8402


Epoch [2/20]: 100%|██████████| 402/402 [01:26<00:00,  4.62it/s, loss=0.000141]


Epoch 2 complete. Average Loss: 0.1256


Epoch [3/20]: 100%|██████████| 402/402 [01:26<00:00,  4.64it/s, loss=0.124] 


...

Epoch 18 complete. Average Loss: 0.0247


Epoch [19/20]: 100%|██████████| 402/402 [02:00<00:00,  3.34it/s, loss=0.0389] 


Epoch 19 complete. Average Loss: 0.0274


Epoch [20/20]: 100%|██████████| 402/402 [01:29<00:00,  4.50it/s, loss=0.0717] 

Epoch 20 complete. Average Loss: 0.0255
Model saved successfully!


In [ ]:
from sklearn.metrics import r2_score, mean_squared_error

test_ds = MultimodalDataset("data/processed_test.csv", "house_images", transform)
test_loader = DataLoader(test_ds, batch_size=32, shuffle=False)

model.eval()
all_preds = []
all_targets = []

with torch.no_grad():
    for images, tabular, targets in tqdm(test_loader, desc="Evaluating"):
        images, tabular = images.to(device), tabular.to(device)
        
        outputs = model(images, tabular)
        
        all_preds.extend(outputs.cpu().numpy())
        all_targets.extend(targets.numpy())

real_preds = np.expm1(all_preds)
real_targets = np.expm1(all_targets)
rmse = np.sqrt(mean_squared_error(real_targets, real_preds))
r2 = r2_score(real_targets, real_preds)

print(f"\n--- Model Performance ---")
print(f"RMSE: ${rmse:,.2f}")
print(f"R² Score: {r2:.4f}")

Evaluating: 100%|██████████| 101/101 [00:56<00:00,  1.79it/s]


--- Model Performance ---
RMSE: $166,183.80
R² Score: 0.7639


In [27]:
import pandas as pd
import numpy as np
import torch
import os
import joblib
from torch.utils.data import DataLoader, Dataset
from PIL import Image
from tqdm import tqdm

# 1. SETUP & LOAD MODEL
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
scaler = joblib.load('data/scaler.pkl')

# Re-initialize the model and load your 75% accuracy weights
model = MultiModalModel(tabular_input_dim=17).to(device)
model.load_state_dict(torch.load("data/multimodal_model.pth"))
model.eval()

# 2. LOAD AND PREPARE TEST2 DATA
test2_df = pd.read_excel("test2.xlsx")

# Same engineering logic as training
test2_df['house_age'] = 2025 - test2_df['yr_built']
test2_df['is_renovated'] = (test2_df['yr_renovated'] > 0).astype(int)
test2_df['sqft_lot_log'] = np.log1p(test2_df['sqft_lot'])

features = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot_log', 'floors', 'waterfront', 
            'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'sqft_living15', 
            'sqft_lot15', 'lat', 'long', 'house_age', 'is_renovated']

# Scale features (use the scaler from training!)
X_test2_scaled = scaler.transform(test2_df[features])

# 3. ROBUST DATASET CLASS
class FinalInferenceDataset(Dataset):
    def __init__(self, df, scaled_features, img_dir, transform):
        self.df = df
        self.features = torch.tensor(scaled_features, dtype=torch.float)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_id = int(self.df.iloc[idx]['id'])
        img_path = os.path.join(self.img_dir, f"{img_id}.jpg")
        
        # Check if file exists to prevent FileNotFoundError
        if os.path.exists(img_path):
            image = Image.open(img_path).convert('RGB')
            if self.transform:
                image = self.transform(image)
        else:
            # Fallback: Blank image (zeros) so the Tabular branch can still work
            image = torch.zeros((3, 224, 224))
            
        return image, self.features[idx], img_id

# 4. PREDICTION LOOP
inference_ds = FinalInferenceDataset(test2_df, X_test2_scaled, "house_images", transform)
inference_loader = DataLoader(inference_ds, batch_size=32, shuffle=False)

all_ids, all_preds = [], []

with torch.no_grad():
    for imgs, tabs, house_ids in tqdm(inference_loader, desc="Predicting 5204 Houses"):
        imgs, tabs = imgs.to(device), tabs.to(device)
        
        outputs = model(imgs, tabs)
        
        # Reverse log transform: exp(x) - 1
        real_prices = np.expm1(outputs.cpu().numpy())
        
        all_preds.extend(real_prices.flatten())
        all_ids.extend(house_ids.numpy())

# 5. SAVE SUBMISSION
submission = pd.DataFrame({'id': all_ids, 'predicted_price': all_preds})
submission.to_csv("final_predictions.csv", index=False)

print(f"Successfully generated {len(submission)} predictions.")

Predicting 5204 Houses: 100%|██████████| 169/169 [00:09<00:00, 17.39it/s]


Successfully generated 5404 predictions.


In [ ]:
import torch.nn.functional as F
import cv2

def generate_gradcam(model, img_tensor, tab_tensor):
    model.eval()
    
    # 1. Target the last convolutional layer in your vision branch 
    target_layer = model.vision_features[-1][-1].bn2 
    
    # 2. Hooks to capture gradients and activations
    gradients = []
    activations = []
    def save_gradient(grad): gradients.append(grad)
    def save_activation(module, input, output): activations.append(output)
    
    target_layer.register_forward_hook(save_activation)
    
    # 3. Forward pass
    output = model(img_tensor.unsqueeze(0), tab_tensor.unsqueeze(0))
    model.zero_grad()
    
    # 4. Backward pass to get gradients
    output.backward()
    target_layer.weight.register_hook(save_gradient)
    
    # 5. Calculate weights and heatmap
    weights = torch.mean(gradients[0], dim=(2, 3), keepdim=True)
    cam = torch.sum(weights * activations[0], dim=1).squeeze().detach().cpu().numpy()
    
    # 6. Normalize and resize
    cam = np.maximum(cam, 0)
    cam = cv2.resize(cam, (224, 224))
    cam = (cam - cam.min()) / (cam.max() - cam.min())
    return cam